In [5]:
import sys
import pandas as pd
import datetime as dt
import os
from flask import Flask,jsonify,request

username = os.getlogin()

hoy = dt.datetime.now()
date_format = "%d.%m.%Y"
date_formatexport = "%d-%m-%Y"
date_formatcartola = "%d/%m/%Y"
fecha_actual = hoy.strftime(date_format)
fecha_cartola = hoy.strftime(date_formatexport)

df1 = pd.DataFrame()
df2 = pd.DataFrame()

# Directorios
folder_path = f'C:\\Users\\{username}\\Documents\\Py\\ARCHIVOS PROCESO AM\\'
folder_filtros = folder_path + 'FILTROS\\'
dir_hist = folder_path + 'CARTOLAS HISTORICAS\\HISTORICA ' + fecha_actual + '.csv'
dir_linea = folder_path + 'CARTOLAS LINEA\\LINEA ' + fecha_actual + '.csv'
dir_merge = folder_path + 'GENERADOS\\MERGE\\MERGE ' + fecha_actual + '.csv'
dir_b2b = folder_path + 'GENERADOS\\B2B\\B2B ' + fecha_actual + '.csv'
dir_fintoc = folder_path + 'GENERADOS\\FINTOC\\FINTOC ' + fecha_actual + '.csv'
dir_tes = folder_path + 'GENERADOS\\TES\\TES ' + fecha_actual + '.csv'
dir_dfcna = folder_path + 'GENERADOS\\NA\\NA ' + fecha_actual + '.csv'
dir_dfcnar = folder_path + 'GENERADOS\\REVERSAR\\REVERSA ' + fecha_actual + '.csv'

cols = ['fecha', 'sucursal', 'operacion', 'descripcion','cargos','abonos']

# Cargar archivos
print("Cargando archivos...")

df_b2b = pd.read_csv(dir_b2b, sep=';', header=None, names=cols, thousands='.',date_format=date_formatexport)
df_fintoc = pd.read_csv(dir_fintoc, sep=';', header=None, names=cols, thousands='.',date_format=date_formatexport)
df_tes = pd.read_csv(dir_tes, sep=';', header=None, names=cols, thousands='.',date_format=date_formatexport)
df_dfcna = pd.read_csv(dir_dfcna, sep=';', header=None, names=cols, thousands='.',date_format=date_formatexport)

# Filtros por glosa exacta
flineadf = pd.read_table(os.path.join(folder_filtros, 'filtroslinea.txt'), header=None, names=['registros'])
fhistdf = pd.read_table(os.path.join(folder_filtros, 'filtroshistorica.txt'), header=None, names=['registros'])
fb2bdf = pd.read_table(os.path.join(folder_filtros, 'filtrosb2b.txt'), header=None, names=['registros'])
ffintocdf = pd.read_table(os.path.join(folder_filtros, 'filtrosfintoc.txt'), header=None, names=['registros'])
flinea = flineadf['registros'].astype(str).tolist()
fhist = fhistdf['registros'].astype(str).tolist()
fb2b = fb2bdf['registros'].astype(str).tolist()
ffintoc = ffintocdf['registros'].astype(str).tolist()

print("OK")

Cargando archivos...
OK


In [7]:
app = Flask(__name__)

@app.route('/cartolabech/abonosb2b',methods=['GET'])    
def get_abonosb2b():
    return jsonify(df_b2b.to_dict(orient='records'))

@app.route('/mp/fintoc',methods=['GET'])    
def get_fintoc():
    return jsonify(df_fintoc.to_dict(orient='records'))

@app.route('/tes/df_tes',methods=['GET'])    
def get_tes():
    return jsonify(df_tes.to_dict(orient='records'))

if __name__ == '__main__':
    app.run(debug=False)

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [24/Jan/2025 12:58:30] "GET /tes/df_tes HTTP/1.1" 200 -
127.0.0.1 - - [24/Jan/2025 12:58:33] "GET /mp/fintoc HTTP/1.1" 200 -
127.0.0.1 - - [24/Jan/2025 12:58:40] "GET /cartolabech/abonosb2b HTTP/1.1" 200 -


In [11]:
dfcnar = dfc.copy()
dfcna = dfc.copy()
for value in dfcna['abonos']:
    counth = dfcna[dfcna['ind'].str.contains("H") & (dfcna['abonos'] == value)].shape[0]
    countl = dfcna[dfcna['ind'].str.contains("L") & (dfcna['abonos'] == value)].shape[0]
    if countl > counth:
        dfcna.drop(dfcna[dfcna['abonos'] == value].index[:counth + countl], inplace=True)
        df2.drop(df2[df2['abonos'] == value].index[:countl], inplace=True)
    else:
        dfcna.drop(dfcna[dfcna['abonos'] == value].index[:countl * 2], inplace=True)
        dfcnar.drop(dfcnar[dfcnar['abonos'] == value].index[:counth + countl], inplace=True)
dfcna[['sucursal_x','operacion_x','descripcion_x','fecha_x']] = dfcna[['sucursal_y','operacion_y','descripcion_y','fecha_y']]
dfcna.rename(columns={'sucursal_x': 'sucursal'}, inplace=True)
dfcna.rename(columns={'operacion_x': 'operacion'}, inplace=True)
dfcna.rename(columns={'descripcion_x': 'descripcion'}, inplace=True)
dfcna.rename(columns={'fecha_x': 'fecha'}, inplace=True)
dfcna = dfcna.drop(columns=['sucursal_y', 'operacion_y', 'descripcion_y', 'fecha_y'])
dfcna = dfcna[~dfcna['abonos'].isin(b2b['abonos'])]
dfcna = dfcna[~dfcna['abonos'].isin(fintoc['abonos'])]
dfcna = dfcna[~dfcna['abonos'].isin(cargos['cargos'])]
dfcna = dfcna[~dfcna['abonos'].isin(rechazo['abonos'])]
dfcna = dfcna[dfcna['abonos'] != 0]
cartola = pd.concat([df2, dfcna]).drop(columns=['ind'])
cartola.to_csv('C:\\Users\\' + username + '\\Documents\\Py\\ARCHIVOS PROCESO AM\\GENERADOS\\'+ file,sep=';',header=False,index=False)